In [ ]:
### Importing the requisite packages

#Data management
import geopandas as gpd
import numpy as np
import pandas as pd

# Visualization
import seaborn
import contextily
import matplotlib.pyplot as plt

# Spatial Statistics
from pointpats import distance_statistics, QStatistic, random, PointPattern, PoissonPointProcess, PoissonClusterPointProcess, Window, poly_from_bbox
import libpysal as ps
from libpysal.cg import shapely_ext

# Setting matplotlib jupyter options to show plots in notebook
%matplotlib inline

In [ ]:
### Reading in the data
data_path = r'YOUR FILE PATH'

In [ ]:
ohio = gpd.read_file(data_path + 'Ohio\Ohio.shp')
ohio_st = ohio.dissolve()
ohio_st

In [ ]:
# Access shapefile of the state of ohio
oh = ps.io.open(data_path + 'Ohio\Ohio.shp')
polys = [shp for shp in oh]

In [ ]:
state = shapely_ext.cascaded_union(polys)

# create window from ohio state boundary
window = Window(state.parts)

In [ ]:
# Here, we're simulating a csr process in the window being Ohio (200 points, 1 realization)
# setting "asPP" True generates a point pattern
# setting "conditioning" to false simulates a N-conditioned CSR
np.random.seed(5)
samples = PoissonPointProcess(window, 200, 1, conditioning=False, asPP=True)

pp_csr = samples.realizations[0] # simulated point pattern

pp_csr.plot(window=True, hull=False, title='Random Point Pattern')

In [ ]:
np.random.seed(10)
csamples = PoissonClusterPointProcess(window, 200, 5, 0.5, 1, asPP=True)
pp_pcp = csamples.realizations[0]
pp_pcp.plot(window=True, hull=False, title='Clustered Point Pattern')

In [ ]:
# Reading in the data from the path
locs_pdf = pd.read_csv(data_path + 'OSM_DollarGeneralLocs.csv')

# Converting the pandas dataframe into a geopandas geodataframe
locs_gdf = gpd.GeoDataFrame(
    locs_pdf, geometry=gpd.points_from_xy(locs_pdf.X, locs_pdf.Y),
    crs="EPSG:4326"
)

In [ ]:
# Create an initial visualization of the data 
f, ax = plt.subplots(1, figsize=(8, 8))
ohio_st.plot(ax = ax,color=None, zorder=1)
locs_gdf.plot(ax = ax, zorder=2, color='black', markersize=8)

In [ ]:
# Create a better visualization
joint_axes = seaborn.jointplot(
    x=locs_gdf.geometry.x, 
    y=locs_gdf.geometry.y, 
    data=locs_gdf, 
    s=5, 
    height=7,
color='k');

contextily.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:4326",
    source=contextily.providers.Stamen.TonerLite
);


## Ripley's Alphabet

### Ripley's G

In [ ]:
g_test = distance_statistics.g_test(
    locs_gdf[['X','Y']].values, support=40, keep_simulations=True
)

In [ ]:
plt.plot(g_test.support, np.median(g_test.simulations, axis=0), 
         color='k', label='Simulated Data')
plt.plot(g_test.support, g_test.statistic, 
         marker='x', color='orangered', label='Observed Data')
plt.legend()
plt.xlabel('Distance')
plt.ylabel('Ripleys G Function')
plt.title('Ripleys G Function Plot')
plt.show()

### Ripley's K

In [ ]:
k_test = distance_statistics.k_test(locs_gdf[['X','Y']].values, keep_simulations=True)

In [ ]:
plt.plot(k_test.support, k_test.simulations.T, color='k', alpha=.01)
plt.plot(k_test.support, k_test.statistic, color='orange')

plt.scatter(k_test.support, k_test.statistic, 
            cmap='viridis', c=k_test.pvalue < .05,
            zorder=4
           )

plt.xlabel('Distance')
plt.ylabel('Ripleys K Function')
plt.title('Ripleys K Function Plot')
plt.show()